In [ ]:
from pathlib import Path

from sed import SedProcessor
import sed
import numpy as np

%matplotlib inline
# %matplotlib ipympl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
config_file = Path(sed.__file__).parent.parent/'tutorial/hextof_config.yaml'
assert config_file.exists()

# Loading Data

In [ ]:
sp = SedProcessor(runs=[44797], config=config_file, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()

# Energy Calibration

## using lmfit

In [ ]:
axes = ['sampleBias', 'dldTimeSteps']
bins = [6, 500]
ranges = [[28,33], [4000, 4800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
sp.load_bias_series(binned_data=res)

In [ ]:
ranges=(4250, 4500)
ref_id=3
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id)

In [ ]:
ref_id=3
ref_energy=0
sp.calibrate_energy_axis(
    ref_id=ref_id, 
    ref_energy=ref_energy,
    method="lmfit",
    energy_scale='kinetic',
    d={'value':1e9, 'min': 1e8, 'max': 1e10},
    E0={'value':0, 'min': -100, 'max': 100,'vary':True},
    t0={'value':225, 'min': -100, 'max': 250, 'vary':True},
)

In [ ]:
sp.append_energy_axis()

In [ ]:
sp.append_tof_ns_axis()

In [ ]:
sp.dataframe[['dldTime','dldTimeSteps','energy','dldSectorID']].head()

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,10]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.mean('sampleBias').plot.line(x='energy',linewidth=3);
res.plot.line(x='energy',linewidth=1,alpha=.5,label='all');


In [ ]:
sp.apply_energy_offset(
    constant=-31.5, 
    columns=['sampleBias'],
    signs=[+1],
)

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-3,2]]
res_fit = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_fit.energy.attrs['unit'] = 'eV'
res_fit.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax);
res_fit.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

## with poly fit

In [ ]:
sp = SedProcessor(runs=[44797], config=config_file, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()

In [ ]:
axes = ['sampleBias', 'dldTimeSteps']
bins = [6, 500]
ranges = [[28,33], [4000, 4800]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
sp.load_bias_series(binned_data=res)

In [ ]:
ranges=(4250, 4500)
ref_id=3
sp.find_bias_peaks(ranges=ranges, ref_id=ref_id)

In [ ]:
ref_id=3
ref_energy=-0.3
sp.calibrate_energy_axis(
    ref_id=ref_id, 
    ref_energy=-0.3, 
    method="lstsq",
    order=2,
)

In [ ]:
sp.append_energy_axis()

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,10]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
res.mean('sampleBias').plot.line(x='energy',linewidth=3);
res.plot.line(x='energy',linewidth=1,alpha=.5,label='all');


In [ ]:
sp.apply_energy_offset(
    constant=-31.5, 
    columns=['sampleBias'],
    signs=[+1],
)

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-3,2]]
res_poly = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_poly.energy.attrs['unit'] = 'eV'
res_poly.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax);
res_poly.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

# From manual values
these are obviously not the right values, I think I got them from an other calibraiton file.
They are here to show how the calibration works.
Also, I noticed the parameter energy_scale from the config has no effect...

In [ ]:
sp = SedProcessor(runs=[44797], config=config_file, collect_metadata=False)

In [ ]:
sp.add_jitter()
sp.align_dld_sectors()

In [ ]:
sp.append_energy_axis()

In [ ]:
sp.apply_energy_offset(
    constant=+31.5, 
    columns=['sampleBias'],
    signs=[-1],
)

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-10,5]]
res_config = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_config.energy.attrs['unit'] = 'eV'
res_config.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax);
res_config.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

In [ ]:
sp.dataframe['energy'] = - sp.dataframe['energy']

In [ ]:
axes = ['sampleBias', 'energy']
bins = [5, 500]
ranges = [[28,33], [-3,2]]
res_config = sp.compute(bins=bins, axes=axes, ranges=ranges)

In [ ]:
plt.figure()
ax = plt.subplot(111)
res_config.energy.attrs['unit'] = 'eV'
res_config.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax);
res_config.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax);

In [ ]:
# compare the two methods
fig, ax = plt.subplots(1,3, figsize=(10,4), layout='constrained')
res_poly.energy.attrs['unit'] = 'eV'
res_poly.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax[0], label='all');
res_poly.plot.line(x='energy',linewidth=1,alpha=.5,ax=ax[0]);
ax[0].set_title('poly')
res_fit.energy.attrs['unit'] = 'eV'
res_fit.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax[1], label='all');
res_fit.plot.line(x='energy',linewidth=1,alpha=.5,ax=ax[1]);
ax[1].set_title('fit')
res_config.energy.attrs['unit'] = 'eV'
res_config.mean('sampleBias').plot.line(x='energy',linewidth=3, ax=ax[2]);
res_config.plot.line(x='energy',linewidth=1,alpha=.5,label='all',ax=ax[2]);
ax[2].set_title('config')
ax[0].set_xlim(-2, 1.5)
ax[1].set_xlim(-2, 1.5)
ax[2].set_xlim(-2, 1.5)